<a href="https://colab.research.google.com/github/yamadashamoji/tools/blob/main/018_hashed_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ハッシュ化ツール

!pip install -q qrcode[pil] ipywidgets

# 必要なライブラリのインポート
import hashlib
import base64
import hmac
import qrcode
from IPython.display import display, clear_output, Image, HTML
import ipywidgets as widgets
from io import BytesIO
from google.colab import files

# ハッシュアルゴリズム一覧
HASH_ALGOS = {
    "MD5": "md5",
    "SHA-1": "sha1",
    "SHA-224": "sha224",
    "SHA-256": "sha256",
    "SHA-384": "sha384",
    "SHA-512": "sha512",
    "SHA3-256": "sha3_256",
    "SHA3-512": "sha3_512"
}

# --- UI要素の作成 ---
input_text = widgets.Textarea(
    placeholder='ここに文字列を入力してください',
    description='入力文字列:',
    layout=widgets.Layout(width='100%', height='100px')
)

algo_dropdown = widgets.Dropdown(
    options=list(HASH_ALGOS.keys()),
    value='SHA-256',
    description='アルゴリズム:'
)

output_format = widgets.RadioButtons(
    options=['Hex（16進数）', 'Base64'],
    value='Hex（16進数）',
    description='出力形式:'
)

use_hmac_checkbox = widgets.Checkbox(
    value=False,
    description='HMACを使用する（鍵付きハッシュ）'
)

hmac_key_input = widgets.Text(
    value='',
    placeholder='HMAC用の鍵を入力',
    description='HMAC鍵:',
    layout=widgets.Layout(width='50%')
)

file_upload = widgets.FileUpload(
    accept='*',
    multiple=False,
    description='ファイルを選択'
)

generate_button = widgets.Button(
    description='ハッシュ生成',
    button_style='success'
)

copy_button = widgets.Button(
    description='コピー',
    button_style='info'
)

output_area = widgets.Output()

# --- ハッシュ生成処理 ---
def generate_hash(b):
    with output_area:
        clear_output()
        is_file = len(file_upload.value) > 0
        text_data = None

        # 入力の取得
        if is_file:
            file_info = next(iter(file_upload.value.values()))
            text_data = file_info['content']
            print(f"📄 ファイル名: {file_info['metadata']['name']}")
        else:
            text_data = input_text.value.encode('utf-8')

        algo = HASH_ALGOS[algo_dropdown.value]

        # HMACまたは通常のハッシュ処理
        if use_hmac_checkbox.value:
            key = hmac_key_input.value.encode('utf-8')
            digest = hmac.new(key, text_data, algo).digest()
        else:
            hash_func = hashlib.new(algo)
            hash_func.update(text_data)
            digest = hash_func.digest()

        # 出力形式の変換
        if output_format.value == 'Hex（16進数）':
            result = digest.hex()
        else:
            result = base64.b64encode(digest).decode('utf-8')

        # 結果の表示
        print(f"🔒 ハッシュ結果（{algo_dropdown.value}）:")
        print(result)
        print("\n📏 情報:")
        print(f"- 入力バイト数: {len(text_data)}")
        print(f"- 出力バイト数: {len(digest)}")

        # QRコードの生成と表示
        qr = qrcode.make(result)
        buf = BytesIO()
        qr.save(buf, format='PNG')
        buf.seek(0)
        display(Image(data=buf.getvalue(), format='png'))

        # JavaScriptでクリップボードにコピーするコードをHTMLで埋め込み
        js_code = f"""
        <script>
        async function copyToClipboard(text) {{
            await navigator.clipboard.writeText(text);
            alert("コピーしました！");
        }}
        </script>
        <button onclick="copyToClipboard('{result}')">結果をコピー</button>
        """
        display(HTML(js_code))

# イベントバインド
generate_button.on_click(generate_hash)

# ウィジェットの表示
display(widgets.VBox([
    input_text,
    file_upload,
    algo_dropdown,
    output_format,
    use_hmac_checkbox,
    hmac_key_input,
    generate_button,
    output_area
]))


In [ ]:
# @title アルゴリズム比較機能

import hashlib
import base64
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# 比較対象のハッシュアルゴリズム
COMPARE_ALGOS = [
    "md5", "sha1", "sha224", "sha256",
    "sha384", "sha512", "sha3_256", "sha3_512"
]

# ウィジェットの作成
compare_input_text = widgets.Textarea(
    placeholder='比較したい文字列をここに入力してください',
    description='入力文字列:',
    layout=widgets.Layout(width='100%', height='100px')
)

compare_output_format = widgets.RadioButtons(
    options=['Hex（16進数）', 'Base64'],
    value='Hex（16進数）',
    description='出力形式:'
)

compare_button = widgets.Button(
    description='比較する',
    button_style='primary'
)

compare_output_area = widgets.Output()

# ハッシュ比較関数
def compare_hashes(b):
    with compare_output_area:
        clear_output()
        input_data = compare_input_text.value.encode('utf-8')
        format_type = compare_output_format.value

        results = []

        for algo in COMPARE_ALGOS:
            hash_func = hashlib.new(algo)
            hash_func.update(input_data)
            digest = hash_func.digest()
            if format_type == 'Hex（16進数）':
                output = digest.hex()
            else:
                output = base64.b64encode(digest).decode('utf-8')
            results.append({
                'アルゴリズム': algo.upper(),
                '出力': output,
                '出力バイト数': len(digest)
            })

        # 結果を表形式で表示
        df = pd.DataFrame(results)
        display(df)

# ボタンにバインド
compare_button.on_click(compare_hashes)

# UI表示
display(widgets.VBox([
    compare_input_text,
    compare_output_format,
    compare_button,
    compare_output_area
]))


## 📄 ハッシュ化ツール 仕様書（Python／Google Colab）

### 🧩 目的
任意の文字列を入力し、指定されたハッシュアルゴリズムでハッシュ化された文字列を生成・表示するツールを作成する。

---

### 📌 使用環境
- **言語**：Python 3.x
- **実行環境**：Google Colab
- **ライブラリ**：
  - 標準ライブラリ：`hashlib`, `base64`, `hmac`, `IPython.display` など
  - 必要に応じて：`cryptography`, `passlib` など（拡張時）

---

### 🖥️ ユーザーインターフェース
Google Colab上の**対話型ウィジェット（`ipywidgets`）**を使って、使いやすいUIを実現する。

#### 入力項目
- 入力文字列（テキストボックス）
- ハッシュアルゴリズムの選択（プルダウンメニュー）
  - `MD5`
  - `SHA-1`
  - `SHA-224`
  - `SHA-256`
  - `SHA-384`
  - `SHA-512`
  - `SHA3-256`
  - `SHA3-512`
- オプション設定（チェックボックスなど）
  - 出力形式：通常の16進文字列 / Base64（オプション）

#### 出力項目
- ハッシュ化された文字列（コピーしやすいように）
- 入力のバイト数・出力のバイト数（参考情報）

---

### 🛠️ 機能一覧

| 機能                          | 説明 |
|-----------------------------|------|
| 入力文字列の取得             | テキストボックスから取得 |
| アルゴリズムの選択           | `hashlib`でサポートされるアルゴリズムを選択 |
| ハッシュの生成               | `hashlib`を使用して安全にハッシュ化 |
| 出力形式の切り替え           | 16進数 or Base64出力 |
| 結果のコピー対応             | `IPython.display`でコピー可能に表示（必要に応じて） |

---

### 🧪 拡張予定機能（任意）

- ハッシュ化だけでなく**HMAC（鍵付きハッシュ）**に対応
- **ファイルのハッシュ化**（アップロードされたファイルを対象に）
- 複数入力の一括処理（CSVなど）
- ハッシュ強度の簡易評価表示
- QRコードで出力（セキュリティ用途）

---

### ✅ 成果物
- Colabノートブック形式のPythonファイル（`.ipynb`）
- 入力・出力に対応したウィジェット付きツール

---